# 思维链智能体

In [178]:
from langchain.prompts import PromptTemplate

_prompt_react = PromptTemplate.from_template("""
你是强大的AI助手，可以使用工具与指令自动化解决问题。

# 你必须遵循以下约束来完成任务:
1. 你只在必要时选择工具，如果可以直接回答，就不要选择任何工具。
2. 每次你的决策只使用一种工具，你可以使用任意多次。
3. 如果你已经完成所有任务，确保以"FINISH"指令结束。
4. 用中文思考和输出。

# 你的任务是:
{input}

# 有两种情况你都应当选择FINISH工具

(1) 如果任务显示“无”、“没有了”、“已完成”、“FINISH”或类似表达，你就选择FINISH工具，
(2) 如果没有选择任何工具就获得了输出，你就直接选择FINISH工具

无论上述哪一种情况，你都必须将最终结果以键值对的方式输出给FINISH工具，其中键的名称是"output"，值就是你的最终结果。

# 如果必须选择工具才能完成任务，你可以使用以下工具之一，它们又称为动作或actions:

{tools}

# 当前的任务执行记录:
{agent_scratchpad}

# 输出形式：
## 首先，根据以下格式说明，输出你的思考过程:
1. 思考: 观察执行记录和你的自我反思，并一步步思考
2. 推理: 根据你的思考，一步步推理解决问题。
3. 计划: 严格遵守以下规则，计划你的当前动作。
  （1）详细列出当前动作的执行计划。只计划一步的动作。PLAN ONE STEP ONLY!
  （2）不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源。不要编造信息。DO NOT MAKE UP ANY INFORMATION!!!
  （3）如果全部子任务已完成，请用FINISH动作结束任务。

## 然后，根据以下格式说明，输出你选择执行的动作/工具:
{action_format_instructions}
""")

In [199]:
from langchain.prompts import PromptTemplate

_prompt_cot = PromptTemplate.from_template("""
你是强大的AI助手，可以使用工具与指令自动化解决问题。

# 你必须遵循以下约束来完成任务:
1. 每次你的决策只使用一种工具，你可以使用任意多次。
2. 确保你调用的指令或使用的工具在下述给定的工具列表中。
3. 确保你的回答不会包含违法或有侵犯性的信息。
4. 如果你已经完成所有任务，确保以"FINISH"指令结束。
5. 用中文思考和输出。
6. 如果执行某个指令或工具失败，尝试改变参数或参数格式再次调用。
7. 你生成的回复必须遵循上文中给定的事实信息。不可以编造信息。DO NOT MAKE UP INFORMATION.
8. 如果得到的结果不正确，尝试更换表达方式。
9. 已经得到的信息，不要反复查询。
10. 确保你生成的动作是可以精确执行的。动作做中可以包括具体方法和目标输出。
11. 看到一个概念时尝试获取它的准确定义，并分析从哪些输入可以得到它的具体取值。
12. 生成一个自然语言查询时，请在查询中包含全部的已知信息。
13. 在执行分析或计算动作前，确保该分析或计算中涉及的所有子概念都已经得到了定义。
14. 你不可以打印一个文件的全部内容，这样的操作代价太大，且会造成不可预期的后果，是被严格禁止的。
15. 不要向用户提问。

# 你的任务是:
{input}

你有非常优秀的逻辑分析能力，可以通过因果关系找到最优的解决方案。

# 有两种情况你都应当选择FINISH工具

(1) 如果任务显示“无”、“没有了”、“已完成”、“FINISH”或类似表达，你就选择FINISH工具，
(2) 如果没有选择任何工具就获得了输出，你就直接选择FINISH工具

无论上述哪一种情况，你都必须将最终结果以键值对的方式输出给FINISH工具，其中键的名称是"output"，值就是你的最终结果。

# 如果必须选择工具才能完成任务，你可以使用以下工具之一，它们又称为动作或actions:

{tools}

# 你需要评估你的表现:
1. 尽你最大的努力，用你最好的水平，通过分析和检查，做出最好的决定。
2. 带着全局观，自我反思你计划与动作。
3. 考虑你之前的策略与决策来改善的你的计划。
4. 如果你反复得到相同的结果，修改你的计划和决策，避免死循环。
5. 如果你当前的动作无法获取到需要的信息，尝试展开关键概念的定义，再重新推理。

# 当前的任务执行记录:
{agent_scratchpad}

# 输出形式：
## 首先，根据以下格式说明，输出你的思考过程:
1. 关键概念: 任务中涉及的组合型概念或实体。已经明确获得取值的关键概念，将其取值完整备注在概念后。
2. 概念拆解: 将任务中的关键概念拆解为一系列待查询的子要素。每个关键概念一行，后接这个概念的子要素，每个子要素一行，行前以' -'开始。
3. 反思:
   - 自我反思，观察以前的执行记录，思考概念拆解是否完整、准确。
   - 一步步思考是否每一个的关键概念或要素的查询都得到了准确的结果。
   - 反思你已经得到哪个要素/概念。你得到的要素/概念取值是否正确。从当前的信息中还不能得到哪些要素/概念。
   - 每个反思一行，行前以' -'开始。
4. 思考: 观察执行记录和你的自我反思，并一步步思考
  （1）分析要素间的依赖关系，例如：
    i. 我是否需要先获得A的值/定义，才能通过A来获得B？
    ii. 如果我先获得A，是否可以通过A筛选B，减少穷举每个B的代价？
    iii. A和B是否存在在同一数据源中，我能否在获取A的同时获取B？
    iv. 是否还有更高效或更聪明的办法来查询一个概念或要素？
    v. 如果上一次尝试查询一个概念或要素时失败了，我是否可以尝试从另一个资源中再次查询？
    vi. 诸如此类，你可以扩展更多的思考 ...
  （2）根据以上分析，排列子要素间的查询优先级
  （3）找出当前需要获得取值的子要素
  注意，不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源！
5. 推理: 根据你的反思与思考，一步步推理被选择的子要素取值的获取方式。如果前一次的计划失败了，请检查输入中是否包含每个概念/要素的明确定义，并尝试细化你的查询描述。
6. 计划: 严格遵守以下规则，计划你的当前动作。
  （1）详细列出当前动作的执行计划。只计划一步的动作。PLAN ONE STEP ONLY!
  （2）一步步分析，包括数据源，对数据源的操作方式，对数据的分析方法。有哪些已知常量可以直接代入此次分析。
  （3）不要尝试计算文件的每一个元素，这种计算代价太高，是严格禁止的。你可以通过分析找到更有效的方法，比如条件筛选。
  （4）上述分析是否依赖某个要素的取值/定义，且该要素的取值/定义尚未获得。若果是，重新规划当前动作，确保所有依赖的要素的取值/定义都已经获得。
  （5）不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源。不要编造信息。DO NOT MAKE UP ANY INFORMATION!!!
  （6）确保你执行的动作涉及的所有要素都已获得确切的取值/定义。
  （7）如果全部子任务已完成，请用FINISH动作结束任务。

## 然后，根据以下格式说明，输出你选择执行的动作/工具:
{action_format_instructions}
""")

In [200]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional, Dict, Any

class Action(BaseModel):
    name: str = Field(description="The name of tool or action: FINISH or Other tool names.")
    args: Optional[Dict[str,Any]] = Field(default=None, description="Parameters of tool or action are composed of names and values.")

In [201]:
from langchain.agents.agent import AgentOutputParser, AgentAction, AgentFinish
from typing import Union
from langchain.output_parsers import PydanticOutputParser

# 解析Action
action_output_parser = PydanticOutputParser(pydantic_object=Action)

class ReasonOutputParser(AgentOutputParser):
    """解析单个动作的智能体action和输入参数。
    """

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        action = action_output_parser.invoke(text)
        name = action.name
        args = action.args if text is not None else "No Args"
        log = text if text is not None else ""
        print("-"*80)
        # print(args)
        print(name)
        # print(log)
        
        if(name == "FINISH"):
            return AgentFinish(args, log)
        elif(name is not None):
            return AgentAction(name, args, log)

    @property
    def _type(self) -> str:
        return "Chain-of-Thought"

In [202]:
from langchain.tools.render import render_text_description_and_args
from langchain_experimental.tools import PythonREPLTool

tools = [PythonREPLTool()]
tools_format = render_text_description_and_args(tools)

action_parser_format = action_output_parser.get_format_instructions()

prompt_react = _prompt_react.partial(
    tools=tools_format,
    action_format_instructions=action_parser_format,
)

prompt_cot = _prompt_cot.partial(
    tools=tools_format,
    action_format_instructions=action_parser_format,
)

In [203]:
print(prompt_react.invoke({"input": "hi", "agent_scratchpad": ""}).text)


你是强大的AI助手，可以使用工具与指令自动化解决问题。

# 你必须遵循以下约束来完成任务:
1. 你只在必要时选择工具，如果可以直接回答，就不要选择任何工具。
2. 每次你的决策只使用一种工具，你可以使用任意多次。
3. 如果你已经完成所有任务，确保以"FINISH"指令结束。
4. 用中文思考和输出。

# 你的任务是:
hi

# 有两种情况你都应当选择FINISH工具

(1) 如果任务显示“无”、“没有了”、“已完成”、“FINISH”或类似表达，你就选择FINISH工具，
(2) 如果没有选择任何工具就获得了输出，你就直接选择FINISH工具

无论上述哪一种情况，你都必须将最终结果以键值对的方式输出给FINISH工具，其中键的名称是"output"，值就是你的最终结果。

# 如果必须选择工具才能完成任务，你可以使用以下工具之一，它们又称为动作或actions:

Python_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`., args: {'query': {'title': 'Query', 'type': 'string'}}

# 当前的任务执行记录:


# 输出形式：
## 首先，根据以下格式说明，输出你的思考过程:
1. 思考: 观察执行记录和你的自我反思，并一步步思考
2. 推理: 根据你的思考，一步步推理解决问题。
3. 计划: 严格遵守以下规则，计划你的当前动作。
  （1）详细列出当前动作的执行计划。只计划一步的动作。PLAN ONE STEP ONLY!
  （2）不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源。不要编造信息。DO NOT MAKE UP ANY INFORMATION!!!
  （3）如果全部子任务已完成，请用FINISH动作结束任务。

## 然后，根据以下格式说明，输出你选择执行的动作/工具:
The output should be formatted as a JSON 

In [227]:
from langchain.agents.format_scratchpad import format_log_to_str
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    temperature=0,
    # model_kwargs={"seed": 42},
)

def create_reason_agent(llm, prompt = None):
    if prompt is None:
        prompt = prompt_react
    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"])
        )
        | prompt
        | llm
        | ReasonOutputParser()
    )
    return agent


In [228]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=create_reason_agent(llm, prompt_cot),
    tools=tools,
    # verbose=True,
    handle_parsing_errors=True
)

In [230]:
question1 = """
    我不知道该怎么设置6位数字密码，请你帮我编写一个计算肥波那契数列的函数，并在前20个数字中随机选择运数字，合成一个数字串，作为我的密码。
    但其中不要包含4，我讨厌这个数字，最终数字串应当是6个数字。
    """

In [229]:
async for chunk in agent_executor.astream_events({"input": question1}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="", flush=True)

### 关键概念:
- 肥波那契数列: 一个数列，其每个数是前两个数的和。此处需要前20个数。
- 数字串: 一个由数字组成的序列，此处需要的是6位数字且不包含数字4。

### 概念拆解:
- 肥波那契数列
  - 数列的定义
  - 生成方法
  - 如何避免包含数字4
- 数字串
  - 从肥波那契数列中选取数字
  - 组合成6位数字串
  - 确保不包含数字4

### 反思:
- 需要确定一个有效的方法来生成肥波那契数列。
- 需要一个筛选机制，确保从数列中选出的数字不包含数字4。
- 应思考一个高效的策略来从前20个肥波那契数中随机选取数字，组成一个6位数字串。

### 思考:
- 先生成肥波那契数列，然后从中筛选不包含数字4的数。
- 确保选出的数字足够多，以便组成多个不同的6位密码。
- 选择一个随机策略来确保密码的随机性和安全性。

### 推理:
- 可以使用Python来实现肥波那契数列的生成和筛选。
- 需要一个函数来生成前20个肥波那契数。
- 另一个函数用来从这些数中筛选出不包含数字4的数。
- 最后，需要一个方法来从筛选后的数中随机选择数字，组成6位数字串。

### 计划:
- 使用Python_REPL工具来编写和执行代码。
- 首先，编写一个函数来生成前20个肥波那契数。
- 其次，编写代码筛选出不包含数字4的数。
- 然后，从筛选后的数字中随机选择6位数字，组成密码。
- 确保所选数字能够组成一个6位的数字串，且每次生成的结果可能不同。

接下来，我将使用Python来实现上述计划。{
  "query": "def generate_fibonacci(n):\n    fib = [0, 1]\n    for i in range(2, n):\n        fib.append(fib[-1] + fib[-2])\n    return fib\n\nfibonacci_20 = generate_fibonacci(20)\nvalid_numbers = [str(num) for num in fibonacci_20 if '4' not in str(num)]\n\nimport random\npassword = ''.join(random.sample(valid_numbers, 6))\

In [231]:
llm_zhipu = ChatZhipuAI(
    model="glm-4",
    temperature=0.1,
)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=create_reason_agent(llm_zhipu, prompt_cot),
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

async for chunk in agent_executor.astream_events({"input": question1}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)



> Entering new AgentExecutor chain...
###_ 思_考_过程_：

**_关键_概念_:**_  
-_ 肥_波_那_契_数_列_  
-_ 随_机_选择_  
-_ 数字_串_  
-_ _6_位_数字_密码_  
-_ 排_除_数字_4_

**_概念_拆_解_:**_  
-_ 肥_波_那_契_数_列_:
 _ -_ -_ 定义_  
 _ -_ -_ 前_20_个_数字_  
-_ 随_机_选择_:
 _ -_ -_ 方法_  
 _ -_ -_ 排_除_特定_数字_（_4_）
-_ 数字_串_:
 _ -_ -_ 长_度_（_6_位_）

**_反思_:**_  
-_ 需_要_明确_肥_波_那_契_数_列_的定义_和_前_20_个_数字_。
-_ 要_确保_随机_选择_逻辑_正确_，_且_排除_数字_4_。
-_ 最终_生成的_数字_串_需要_是_6_位_。

**_思考_:**_  
-_ 首先_，_我_需要_生成_一个_肥_波_那_契_数_列_的前_20_个_数字_，_然后_从中_随机_选择_，_排除_数字_4_。
-_ 可以_使用_Python_的_random_库_来_随机_选择_数字_。
-_ 生_成的_数字_串_需要_恰好_是_6_位_。

**_推理_:**_  
-_ 我_将_使用_Python__RE_PL_工具_编写_一个_函数_来_生成_肥_波_那_契_数_列_，_并_从中_随机_选择_数字_，_直到_生成_一个_6_位_数字_串_，_且_不_包含_数字_4_。

**_计划_:**_  
-_ 使用_Python__RE_PL_执行_以下_步骤_：
 _ _1_._ 定义_肥_波_那_契_数_列_生成_函数_。
 _ _2_._ _生成_前_20_个_肥_波_那_契_数_。
 _ _3_._ 编_写_逻辑_以_随机_选择_数字_，_并_确保_不_选择_数字_4_。
 _ _4_._ _重复_随机_选择_直到_得到_一个_6_位_数字_串_。

###_ 执行_动作_：

```_json_
{
 _ "_name_":_ "_Python__RE_PL_",
 _ "_args_":_ {
   _ "_query_":_ "_def_ fibonacci_(n_):_ a_,_ b_ =

In [218]:
async for chunk in agent_executor.astream_events({
    "input": "写一个关于程序员的笑话"
}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="|", flush=True)
    # else:
    #     print(event)




> Entering new AgentExecutor chain...
###| 思|考|过程|：

**|关键|概念|:**| 程|序|员|笑话|

**|概念|拆|解|:|**
-| 程|序|员|笑话|
 | -| -| 程|序|员|文化|
 | -| -| 笑|话|结构|

**|反思|:|**
-| -| 需|要|确保|笑话|内容|符合|程序员|文化|，|且|格式|符合|笑话|的一般|结构|。

**|思考|:|**
-| 分析|要素|间的|依赖|关系|：
 | -| -| 程|序|员|文化|可以通过|网络|资源|了解|，|笑话|结构|则|依赖于|文学|创作|技巧|。
 | -| -| 可以|先|通过网络|资源|了解|程序员|文化|，|之后|创作|一个|符合|笑话|结构的|笑话|。
-| 排|列|子|要素|间的|查询|优先|级|：
 | -| -| 首|先|了解|程序员|文化|。
 | -| -| 其次|创作|笑话|。
-| 找|出|当前|需要|获得|取|值的|子|要素|：
 | -| -| 程|序|员|文化|。

**|推理|:**| 
-| 根|据|反思|和|思考|，|我|计划|先|通过网络|资源|了解|程序员|文化|，|然后|结合|笑话|的一般|结构|创作|一个|笑话|。

**|计划|:|**
-| 当前|动作|的|执行|计划|：
 | -| -| 利用|网络|资源|了解|程序员|文化|。
 | -| -| 结合|程序员|文化和|笑话|结构|创作|笑话|。
 | -| -| 确|保|笑话|内容|不|违法|、|不|侵犯|他人|权益|。

###| 执行|动作|：

由于|创作|一个|笑话|不需要|使用|工具|，|且|不|违反|任何|约束|，|我将|直接|输出|结果|。

```|json|
{"|name|":| "|FIN|ISH|",| "|args|":| {"|output|":| "|为什么|程序员|总是|携带|电脑|？|因为他们|不想|被人|称为|‘|裸|奔|’|程序员|。"|}}
```| 

输出|笑话|后|，|任务|完成|。||--------------------------------------------------------------------------------
FINISH
### 思考过程：

**